In [74]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from rdkit.Chem import PandasTools, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn import pipeline as pl
from sklearn import preprocessing as pp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import graphviz 


ModuleNotFoundError: No module named 'graphviz'

### Dataset initiation
Initiate and prepare the dataset with descriptors to make them usable for the ML-model. 

In [68]:
dataset_molecules = pd.read_csv('tested_molecules-1.csv')
PandasTools.AddMoleculeColumnToFrame(dataset_molecules, smilesCol='SMILES')
dataset_molecules.head()


,SMILES,ALDH1_inhibition,ROMol
0,COc1ccccc1CC(NC(C)=O)C(=O)NC1CCN(c2nnnn2-c2ccc...,1,<rdkit.Chem.rdchem.Mol object at 0x0000022B596...
1,O=C(CSc1nc2cccnc2n1Cc1ccccc1)NCc1ccco1,1,<rdkit.Chem.rdchem.Mol object at 0x0000022B596...
2,Cc1cccc2cc(C[NH+](CC3CCCO3)C(c3nnnn3Cc3ccco3)C...,1,<rdkit.Chem.rdchem.Mol object at 0x0000022B596...
3,CCN(CC)c1ccc2c(Cl)c(Br)c(=O)oc2c1,1,<rdkit.Chem.rdchem.Mol object at 0x0000022B596...
4,CS(=O)(=O)N1CCc2cc(-c3csc(NC(=O)Cc4cccs4)n3)ccc21,1,<rdkit.Chem.rdchem.Mol object at 0x0000022B596...


In [69]:

descriptors_labels = [n[0] for n in Descriptors._descList[:]]
calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptors_labels)
rdkit_desc = [calc.CalcDescriptors(m) for m in dataset_molecules["ROMol"]]
dataset_descriptors = pd.DataFrame(rdkit_desc, columns=[descriptors_labels])
X_train, X_test, y_train, y_test = train_test_split(dataset_descriptors, dataset_molecules['ALDH1_inhibition'], test_size=0.2, random_state=42)
dataset_descriptors.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,13.083531,13.083531,0.001173,-0.683140,0.520365,463.542,434.310,463.233188,178,0,...,0,0,0,0,1,0,0,0,0,0
1,12.170097,12.170097,0.066966,-0.066966,0.498564,378.457,360.313,378.115047,136,0,...,1,0,0,0,0,0,0,0,0,0
2,10.905837,10.905837,0.016881,-0.016881,0.382043,477.589,444.325,477.260865,184,0,...,0,0,0,0,1,0,0,0,0,0
3,11.562446,11.562446,0.270607,-0.454447,0.795948,330.609,317.505,328.981818,96,0,...,0,0,0,0,0,0,0,0,0,0
4,12.108866,12.108866,0.086947,-3.251317,0.687618,419.553,402.417,419.043204,140,0,...,0,1,0,0,0,1,0,1,0,0


### Introducing a Pipeline with minmax scaler and Desciscion tree.

In [70]:
scaler = pp.MinMaxScaler()
pca = PCA()
dtree = tree.DecisionTreeClassifier(max_features=15)

pipeline = pl.Pipeline(steps=[("sc", scaler), ("pca", pca), ("dtreeCLF", dtree)])
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)


0.585

### Checking which Features are used 

In [73]:
dot_data = tree.export_graphviz(dtree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("descision_tree") 

NameError: name 'graphviz' is not defined

### Analysing the model

In [72]:
y_true = y_test
y_pred = pipeline.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn+fp)
print(tn, fp, fn, tp)
print(specificity)

97 43 40 20
0.6928571428571428
